In [1]:
__file__='/home/ogangza/path-finding-rl/wingHwi/'

In [2]:
from string import ascii_uppercase
import numpy as np
import pandas as pd
import os
import random
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

###################################
# 보상 Reward
move_reward = -1  # 0.1
obs_reward = -10  # 0.1
goal_reward = 10  # 10
###################################
# 그리드값
# target_gridval = 10, 9, 8... # 목표물: 10, 9, 8, ...
curloc_gridval  = 1            # 현 위치: 1
default_gridval = 0            # 기본: 0
rack_gridval    = -1           # 선반: -1
obs_gridval     = -10          # 장애물: -10
###################################
# train / test 모드 지정
train_mode = False
###################################
print('reward:' , move_reward, obs_reward, goal_reward)
PATH = __file__ + '/hoPara_Baseline_DDQN_v1.0.pt'

#__file__ = '/home/ogangza/heung_path_finding/path-finding-rl/data'

local_path = os.path.abspath(os.path.join(os.path.dirname(__file__)))

class Simulator:
    def __init__(self):
        # Load train or test data
        if train_mode:  # 훈련 데이타 읽기
            self.files = pd.read_csv(os.path.join(local_path, "data/factory_order_train.csv"))
            print('data/factory_order_train.csv used')
        else:           # 테스트 데이터 읽기
            self.files = pd.read_csv(os.path.join(local_path, "data/factory_order_test.csv"))
            print('data/factory_order_test.csv used')
        ##########################################################################################
        self.height = 10  # 그리드 높이
        self.width = 9    #  그리드 너비
        self.inds = list(ascii_uppercase)[:17]  # A ~ Q alphabet list

    def set_box(self):  # 선반 위치, 목적지 그리드값 설정. 목적지 리스트 생성
        box_data = pd.read_csv(os.path.join(local_path, "data/box.csv"))
        for box in box_data.itertuples(index = True, name ='Pandas'):  # 선반 위치: 그리드값 = rack_gridval
            self.grid[getattr(box, "row")][getattr(box, "col")] = rack_gridval
        order_item = list(set(self.inds) & set(self.items))
        order_csv = box_data[box_data['item'].isin(order_item)]
        for i, order_box in enumerate(order_csv.itertuples(index = True, name ='Pandas')):  # 목적지: 그리드값 = target_gridval
            self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = 10 - i
            self.local_target.append([getattr(order_box, "row"), getattr(order_box, "col")])  # 목적지 리스트 생성: local_target
        # self.local_target.sort() # 불필요. 인풋 데이터 A, B, C순 정렬되어 있음... 정렬 필요시 코드 바꾸어야 함
        self.local_target.append([9,4]) # 목적지 리스트에 최종 목적지(9,4) 추가
        self.local_target_original = self.local_target.copy()  # gif 생성을 위해 추가.  에피소드의 경로 저장
        self.target_length = len(self.local_target_original)  # 목적지 그리드값 10, 9, 8, ... 지정 위해 추가
        self.grid[9,4] = 10 + 1 - self.target_length  # 최종 목적지 [9,4] 그리드값 지정 위해 추가

    def set_obstacle(self):  # 장애물 위치 그리드값 설정 = obs_gridval
        obstacles_data = pd.read_csv(os.path.join(local_path, "data/obstacles.csv"))
        for obstacle in obstacles_data.itertuples(index = True, name ='Pandas'):
            self.grid[getattr(obstacle, "row")][getattr(obstacle, "col")] = obs_gridval

    ########################################################################################################
    '''
    writer 신휘정
    step에서 아이템을 먹으면 self.items의 첫번째 요소를 pop해줍니다.
    그리고 새로운 item에 대해서 grid_box를 그려줍니다.
    '''
    ########################################################################################################
    def grid_box(self):  # 그리드 박스 초기화 용도로 정의: 선반, 목적지, 장애물 그리드값 초기화
        box_data = pd.read_csv(os.path.join(local_path, "data/box.csv"))
        for box in box_data.itertuples(index = True, name ='Pandas'):  # 선반 위치: 그리드값 = rack_gridval
            self.grid[getattr(box, "row")][getattr(box, "col")] = rack_gridval
        # order_item = list(set(self.inds) & set(self.items))
        if len(self.items) > 0:
            order_item = list(self.items)
            order_csv = box_data[box_data['item'].isin(order_item)]
            for i, order_box in enumerate(order_csv.itertuples(index = True, name ='Pandas')):  # 목적지: 그리드값 = target_gridval
                self.grid[getattr(order_box, "row")][getattr(order_box, "col")] = 10 - i
            self.grid[9,4] = 10 + 1 - self.target_length  # 최종 목적지 [9,4] 그리드값 지정 위해 추가
        else : 
            self.grid[9,4] = 10
        self.set_obstacle()  # 장애물 위치 그리드값 설정 = obs_gridval
        
    def reset(self, epi):  # 에피소드 시작시 12개 값 초기화
        self.epi = epi                                            #1. 에피소드 번호 받기
        self.items = eval(list(self.files.iloc[self.epi])[0])           #2. 에피소드의 목적지 받기: [ 'H', 'L', 'M']
        #####################################################################################
        # coin = random.randint(0,16)
        # self.items = [chr(coin+65)]  # A~Q 까지 1개 랜덤으로 받아서 items에 1개 넣어주기, 반복 횟수는 main의 epochs 조절하기
#         self.items = [chr(epi+65)]
        ####################################################################################
        self.cumulative_reward = 0                                #3. 누적 보상값 = 0
        self.terminal_location = None                             #4. 최초 목적지
        self.local_target = []                                    #5. 목적지 리스트 초기화
        self.actions = []                                         #6. 지나온 경로 리스트 초기화
        self.grid = np.zeros((self.height, self.width), dtype="float16")  #7. 그리드월드 전체 그리드값(default_gridval) 초기화
        self.set_box()                                            #8. 선반 위치, 목적지 그리드값 설정. 목적지 리스트 생성
        self.set_obstacle()                                       #9. 장애물 그리드값 설정
        ######################
        # print('최초 그리드맵:')  # 그리드맵 확인
        # print(self.grid.reshape(10,9))
        ######################
        self.curloc = [9, 4]                                      #10. 현재 위치를 출발점으로 세팅
        self.grid[int(self.curloc[0])][int(self.curloc[1])] = curloc_gridval  #11. 현재 위치(출발점) 그리드값 세팅
        self.done = False                                         #12. 경로 찾기 종료 여부 = False
        self.goal_ob_reward = False                      # (추가) #13. 최종 목적지 도착 여부 = False
        ######################
        # print('현 위치 추가:')   # 그리드맵 확인
        # print(self.grid.reshape(10,9))
        ######################
        self.move_track = [[9,4]]                                  #14. 이동하는 경로를 저장하는 변수, 에피소드 시작시 에이전트가 [9,4]에 있으므로 [[9,4]]로 초기화
        self.isReset =  False                                      #15. 처음으로 terminal_location == 9,4가 되면, moveTrack을 초기화해주기 위해 필요한 변수 isFinal -> isReset 로 변경(처음으로 불러졌냐 안불러졌냐)
        self.prior_distance = 9999                                 #16. 이전 time step의 거리와 현재 거리를 비교하기 위해서 필요한 변수
        ######################
        self.wasGoal = False                                       # 17. 이전 step이 Goal이었는지 확인하는 변수
                                                                   # 이전 step이 Goal이었으면 먹고 나가야 하는 곳이 move_track이 곂치기 때문에 -보상받음. 이를 방지해기위한 FLAG
        return self.grid

    def apply_action(self, action, cur_x, cur_y):  # action에 따른 새 좌표값 반환
        new_x = cur_x
        new_y = cur_y
        if action == 0:          # up
            #new_x = cur_x - 1
            new_x = self.move_up(cur_x, cur_y, new_x, new_y)
        elif action == 1:        # down
            #new_x = cur_x + 1
            new_x = self.move_down(cur_x, cur_y, new_x, new_y)
        elif action == 2:        # left
            #new_y = cur_y - 1
            new_y = self.move_left(cur_x, cur_y, new_x, new_y)
        else:                    # right
            #new_y = cur_y + 1
            new_y = self.move_right(cur_x, cur_y, new_x, new_y)
        return int(new_x), int(new_y)
    
    # >>> 현재 위치에서 이동이 불가한 위치 추가 (시작) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    def move_up(self, cur_x, cur_y, new_x, new_y):  # action == 0:
        if cur_x in [6,5,4,3,2] and cur_y in [0,8]:
            pass
        else:
            new_x = cur_x - 1
        return new_x
    def move_down(self, cur_x, cur_y, new_x, new_y): # action == 1:
        if cur_x in [1,2,3,4,5] and cur_y in [0,8]:
            pass
        else:
            new_x = cur_x + 1
        return new_x
    def move_left(self, cur_x, cur_y, new_x, new_y): # left elif action == 2:
        if cur_y in [1,2,3,4,5,6,7,8] and cur_x == 0:
            pass
        else:
            new_y = cur_y - 1
        return new_y
    def move_right(self, cur_x, cur_y, new_x, new_y): # right else: action == 3:
        if cur_y in [0,1,2,3,4,5,6,7] and cur_x == 0:
            pass
        else:
            new_y = cur_y + 1
        return new_y
    # >>> 추가 (끝) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    def get_reward(self, new_x, new_y, out_of_boundary):  # action에 의해 이동시 얻는 보상
        if any(out_of_boundary):                                        # 바깥으로 나가는 경우
            reward = obs_reward
        else:
            if self.grid[new_x][new_y] == obs_gridval :     # 장애물에 부딪히는 경우
                reward = obs_reward
                self.grid[new_x][new_y] = curloc_gridval   # 현 위치(장애물)에서 종료되었음을 표시. 원래는 장애물이었음                
            elif self.grid[new_x][new_y] == rack_gridval:   # 선반에 부딪히는 경우
                reward = obs_reward
                new_x = self.curloc[0]  # 선반인 경우 못들어가고, 현재 위치로 유지
                new_y = self.curloc[1]  # 선반인 경우 못들어가고, 현재 위치로 유지
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:  # 현재 목적지에 도달한 경우
                reward = goal_reward
                self.prior_distance = 9999                  # 목적지에 갈 때 prior_distance = 1이라, 나갈 때 무조건 prior_distance가 작아서 negative_reward를 받기때문에 여기서 못받도록 수정해줌.
            # 그냥 움직이는 경우 
            else: 
                cur_distance = self.get_distance()
                if self.prior_distance > cur_distance :                         # 이전의 거리 > 현재의 거리 = 현재 아이템에 더 가깝다 => + 보상얻기
                    reward = self.get_positive_distance_reward(cur_distance)
                else:                                                           # 이전의 거리 < 현재의 거리 = 현재 아이템에서 멀어졌다 => - 보상얻기
                    reward = self.get_negative_distance_reward(cur_distance)
                self.prior_distance = cur_distance      

                if [new_x, new_y] not in self.move_track:                           # 이동한 곳이 처음온 곳이면 move_track에 추가
                    self.move_track.append([new_x, new_y])
                else:                                                           # 이동한 곳이 이미 왔던 곳이면 기존 보상에서 -1
                    reward = reward -1
        return reward

    def step(self, action):  # action에 따른 이동 실행
        self.terminal_location = self.local_target[0]           # 목적지 리스트의 첫 번째 요소를 목적지로 설정
        cur_x, cur_y = self.curloc                              # 현재 위치 기억
        new_x, new_y = self.apply_action(action, cur_x, cur_y)  # 다음 위치 받기
        self.actions.append((cur_x, cur_y))                     # 현재 위치(지나온 위치)를 경로 리스트에 추가
        self.goal_ob_reward = False                             # 최종 목적지에 도착한 경우에 True. self.reset()에서 초기화
        out_of_boundary = [new_x < 0, new_x >= self.height, new_y < 0, new_y >= self.width]  # 그리드월드 밖으로 나갔는가? OB = True

        if any(out_of_boundary):  #1. 바깥으로 나가는 경우, 빈 선반에 부딪치는 경우 종료하지 않음
            #print('OB')
            new_x = cur_x  # 종료 처리할 경우 코멘트 처리
            new_y = cur_y  # 종료 처리할 경우 코멘트 처리
            ######################################################종료 처리할 경우 실행
            #self.done = True                             # while loop 종료
            #self.grid_box()                              # 선반, 목적지, 장애물 그리드값 초기화
            #self.actions.append((new_x,new_y))           # 경로 리스트에 최종 위치 추가. (주의) 좌표값이 그리드월드를 벗어나는 값 발생!
            ######################################################
        else:
            if self.grid[new_x][new_y] == obs_gridval:  #2. 장애물에 부딪히는 경우 종료
                # print('장애물')
                # new_x = cur_x  # 종료 처리할 경우 코멘트 처리
                # new_y = cur_y  # 종료 처리할 경우 코멘트 처리
                ###################################################종료 처리할 경우 실행
                self.done = True                           # while loop 종료
                self.grid[cur_x][cur_y] = default_gridval  # 현 위치의 그리드값을 기본값으로 초기화
                self.grid_box()                            # 선반, 목적지, 장애물 그리드값 초기화
                self.actions.append((new_x,new_y))         # 경로 리스트에 최종 위치 추가
                ###################################################
            elif self.grid[new_x][new_y] == rack_gridval:
                pass
            elif new_x == self.terminal_location[0] and new_y == self.terminal_location[1]:  #3. 현재 목적지에 도착한 경우 다음 목적지 설정
                #print('목적지 도착')
                if [new_x, new_y] == [9, 4]:            #3-1. 최종 목적지에 도착한 경우 종료
                    self.done = True                       # while loop 종료
                    self.goal_ob_reward = True             # True = 1 (OB, 장애물 종료 처리 할 경우 self.done=True가 많아지므로 이때 사용)
                    self.actions.append((new_x,new_y))     # 경로 리스트에 최종 위치 추가
                ########################################################################################################
                '''
                writer 신휘정
                기존 : 아이템을 먹어도 에피소드 동안 목적지, 선반 숫자는 유지됨.
                현재 : 아이템을 먹으면 남은 아이템에 대해서 숫자가 새로 칠해짐
                '''
                if len(self.items) > 0:
                    self.items.remove(self.items[0])            # 아이템을 하나먹었으니, 아이템의 첫번째 값을 삭제해줌
                    self.wasGoal = True
                    self.move_track.pop()                       # 아이템을 들어가고 나오는 위치는 동일(주황색동그라미) => 이미 왔던경로라고 - 리워드 못받게 하기위해 마지막 요소 pop()
                ########################################################################################################                    

                self.local_target.remove(self.local_target[0])  # 다음 목적지 설정. 최종 목적지에 도착한 경우에는 마지막 요소인 [9,4]를  제거
                self.grid[cur_x][cur_y] = default_gridval  # 현 위치의 그리드값을 기본값으로 초기화
                self.grid_box()                            # 선반, 목적지, 장애물 그리드값 초기화
                self.grid[new_x][new_y] = curloc_gridval   # 현 위치(목적지)에서 종료되었음을 표시. 원래는 목적지이었음
                self.curloc = [new_x, new_y]               # 새로 도착한 위치를 현재 위치로 변경
            
            else:                                       #4. 그냥 움직이는 경우
                #print('이동')
                ########################################################################################################
                '''
                writer 신휘정
                기존 : 아이템을 먹어도 에피소드 동안 목적지, 선반 숫자는 유지됨.
                현재 : 아이템을 먹으면 해당 선반은 빈선반으로 바뀜
                '''
                if self.wasGoal:
                    self.grid[cur_x][cur_y] = rack_gridval     # 현 위치의 그리드값을 선반값으로 초기화
                    self.wasGoal = False
                else:
                    self.grid[cur_x][cur_y] = default_gridval  # 현 위치의 그리드값을 기본값으로 초기화
                ########################################################################################################
                self.grid_box()                            # 선반, 목적지, 장애물 그리드값 초기화
                self.grid[new_x][new_y] = curloc_gridval   # 현 위치에서 종료되었음을 표시
                self.curloc = [new_x,new_y]

        ############################### 보상받는 처리 시작 ###############################
        # 마지막 위치에서 위로 올라가는 행동을 한다면 - 보상받음 태양
        if len(self.local_target) == 1:                             # local_target이 remove되고 나면 길이가 1이되는 경우 = [9,4]만 있는 경우
            if not self.isReset:                                    # 처음으로 terminal_location == [9,4]가 되었다면,,
                self.move_track = []                                # 아이템을 찾으러 돌아다니는 경로 초기화
                self.isReset = True                                 # move_track이 초기화 되었으니 isReset = True
            reward = self.get_reward(new_x, new_y, out_of_boundary)
            if action == 0:
                reward = obs_reward                                # terminal_location == [9,4]로 갈 때 위로 올라가는거 금지
        else : 
            reward = self.get_reward(new_x, new_y, out_of_boundary)
        ############################### 보상받는 처리 끝 ###############################   

        self.cumulative_reward += reward
        return self.grid, reward, self.cumulative_reward, self.done

    # >>> distance를 입력으로 받아 + 보상을 계산하는 메소드 (시작) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 
    def get_positive_distance_reward(self, distance):
        positive_reward = (20 - distance) / 200     # 0.095 ~ 0.001
        return positive_reward
    # >>> 추가 (끝) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

    # >>> distance를 입력으로 받아 - 보상을 계산하는 메소드 (시작) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    def get_negative_distance_reward(self, distance):
        negative_reward = -(distance / 200)
        return negative_reward
    # >>> 추가 (끝) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<    

    # >>> 현재 위치에서 타겟까지의 거리를 구하는 메소드 (시작) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    def get_distance(self): 
        cur_x, cur_y, target_x, target_y = self.get_current_state()        
        distance = (abs(cur_x-target_x) + abs(cur_y-target_y))
        # print(cur_x, cur_y, target_x, target_y)
        # print('distance : ', distance)
        return distance
    # >>> 추가 (끝) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<    

    # >>> 현재 상황에 대한 정보를 구하는 메소드 (시작) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    def get_current_state(self):
        # if len(self.local_target) != 1:
        #     state = (self.curloc[0], self.curloc[1], self.local_target[0][0], self.local_target[0][1])
        # else :
        state = (self.curloc[0], self.curloc[1], self.terminal_location[0], self.terminal_location[1])
        return np.array(state) #(cur_x, cur_y, tar_x, tar_y)
    # >>> 추가 (끝) <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<        


reward: -1 -10 10


In [3]:
import collections
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ReplayBuffer():  # 리플레이 버퍼: 경험 저장소
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):  # 리플레이 버퍼(메모리)를 경험(transition)으로 채우기
        self.buffer.append(transition)
    
    def sample(self, n):  # memory.sample(batch_size)로 사용
        mini_batch = random.sample(self.buffer, n)  # 미니배치 샘플링
        '''
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        for transition in mini_batch:  # 경험의 각 요소들을 각각의 미니배치로 구성
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
        '''
        state1_batch = torch.cat([s1 for (s1,a,r,s2,d) in mini_batch])
        action_batch = torch.Tensor([[a] for (s1,a,r,s2,d) in mini_batch])      # a를 tensor에 넣을 때는 [리스트]에 꼭 씌워주세요
        reward_batch = torch.Tensor([[r] for (s1,a,r,s2,d) in mini_batch])      # reward를 tensor에 넣을 때는 [리스트]에 꼭 씌워주세요
        state2_batch = torch.cat([s2 for (s1,a,r,s2,d) in mini_batch])
        done_batch = torch.Tensor([[d] for (s1,a,r,s2,d) in mini_batch])        # done를 tensor에 넣을 때는 [리스트]에 꼭 씌워주세요

        return state1_batch, action_batch, reward_batch, state2_batch, done_batch
    
    def size(self):  # 메모리 크기
        return len(self.buffer)

class Qnet(nn.Module):
    '''##### Linear 모델 #####
    def __init__(self):
        super(Qnet, self).__init__()

        in_size = 90  # input 크기
        L1 = 250
        L2 = 150
        out_size  = 4

        self.fc1 = nn.Linear(in_size, L1)
        self.fc2 = nn.Linear(L1, L2)
        self.fc3 = nn.Linear(L2, out_size)

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    '''
    ##### Convolution 모델 #####
    def __init__(self):
        super(Qnet, self).__init__()

        self.conv1 = nn.Conv2d(4, 32, 3, 1) #### 1이었던 부분을 4로 변경
        self.conv2 = nn.Conv2d(32, 64, 3, 1) #### in_채널, out_channel, kernel_size, stride
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 4)

    def forward(self, x):
        x = x.to(device)               # (N, 4, 10, 9)
        x = F.relu(self.conv1(x))      # (N, 32, 8, 7)
        x = F.relu(self.conv2(x))      # (N, 64, 6, 5)
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)        # (N, 64, 30)
        x = F.relu(self.fc1(x))        # (N, 1920) -> (N, 128)
        x = self.fc2(x)                # (N, 128)  -> (N, 4)
        return x

    def sample_action(self, obs, epsilon):  # Qnet()을 실행해서 모든 action에 대한 Qval을 구한 후 action 선택
        out = self.forward(obs)  # out = Q_value, obs = state = 그리드맵
        coin = random.random()  # e-greedy로 action 선택
        if coin < epsilon:
            return random.randint(0,3)
        else : 
            return out.argmax().item()
    
    def test_action(self, obs):  # Qnet()을 실행해서 모든 action에 대한 Qvalue을 구한 후 action 선택
        out = self.forward(obs)  # out = Q_value, obs = state = 그리드맵
        return out.argmax().item()

##################################################### DOUBLE _ DQN ######################################################       
def train(q, q_target, memory, optimizer):  # 메모리에 경험이 미니배치 크기 이상 쌓이면 미니배치 훈련 시작
    s,a,r,s_prime,done_mask = memory.sample(batch_size)  # 메모리에서 미니배치를 뽑기: batch_size=1600
    
    q_out = q(s)  # Qnet의 Qvalue
    q_a = q_out.gather(dim=1,index=a.long().to(device)) # Qnet의 Qvalue 변환

    with torch.no_grad():
        q_prime_idx = q(s_prime).max(1)[1]                                      # output : tensor (8)
        max_q_primes = q_target(s_prime)                                        # output : tensor (8, 4)
        max_q_prime = max_q_primes.gather(1, q_prime_idx.unsqueeze(1))          # output : tensor (8, 1)

    Y = r.to(device) + gamma * max_q_prime * done_mask.to(device)               # output : tensor (8, 1)

    loss = F.smooth_l1_loss(q_a, Y.detach())
    #loss = nn.MSELoss(X.to(torch.float32), Y.detach().to(torch.float32))  # 손실함수 변경. target.detach()로 변경
    optimizer.zero_grad()  # gradient값 초기화
    loss.backward()  # 자동미분
    optimizer.step()  # gradient 업데이트
    return loss.item()
##################################################### DOUBLE _ DQN ######################################################       

########################################################## DQN ##########################################################
# def train(q, q_target, memory, optimizer):  # 메모리에 경험이 미니배치 크기 이상 쌓이면 미니배치 훈련 시작
#     '''
#     for i in range(10):
#         s, a, r, cr, s_prime, done_mask, gr = memory.sample(batch_size)
#         q_out = q(s)
#         q_a = q_out.gather(1,a.to(device))
        
#         max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
#         target = r.to(device) + gamma * max_q_prime * done_mask.to(device)
        
#         loss = F.smooth_l1_loss(q_a, target)
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#     '''
    # s,a,r,s_prime,done_mask = memory.sample(batch_size)  # 메모리에서 미니배치를 뽑기: batch_size=1600
    # Q1 = q(s)  # Qnet의 Qvalue
    # X = Q1.gather(dim=1,index=a.long().unsqueeze(dim=1).to(device)).squeeze() # Qnet의 Qvalue 변환
    # with torch.no_grad():
    #     Q2 = q_target(s_prime)
    # Y = r.to(device) + gamma * done_mask.to(device) * torch.max(Q2,dim=1)[0]
    # loss = F.smooth_l1_loss(X, Y.detach())
    # #loss = nn.MSELoss(X.to(torch.float32), Y.detach().to(torch.float32))  # 손실함수 변경. target.detach()로 변경
    # optimizer.zero_grad()  # gradient값 초기화
    # loss.backward()  # 자동미분
    # optimizer.step()  # gradient 업데이트
    # return loss.item()
########################################################## DQN ##########################################################

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU를 사용할 경우 ???
print('tensor in', device)  # GPU 사용 확인
## test 코드 #################################################################################
def test(epochs=1000, train_mode=False, display=True, model_load=True, PATH=PATH):  # train_mode = False 테스트 모드

    test_env = Simulator()  # 데이터셋 읽기. 그리드월드 크기 지정
    test_q = Qnet()
    test_q.to(device)
    learning_rate = 0.00025    #3. 0.0005
    optimizer = optim.Adam(test_q.parameters(), lr=learning_rate)

    if model_load:
        ###############################################################
        ## 학습한 모델 불러오기
        ###############################################################
        checkpoint = torch.load(PATH)
        test_q.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        test_q.eval()

    ### 액션 저장하는 txt파일 만들기 #######################
    f = open(__file__ + '/hoPara_Baseline_DDQN_v1.0_TEST' + '.txt', 'w')
    ########################################################
        
    #epochs = len(test_env.files)
    wins = 0

    for n_epi in range(epochs):
#         print('new_EPI #################################################')
        gridmap = test_env.reset(n_epi)  # 에피소드 시작시 12개 값 초기화
        history_ = np.stack((gridmap,gridmap,gridmap,gridmap), axis = 0) # 초기 state로 히스토리 초기화 (4,10,9)
        history = np.reshape([history_], (1,4,10,9)) # (배치, 채널, 가로, 세로) 로 reshape
        history = torch.from_numpy(history).float() # 텐서로 변환   
        done = False
        mov = 0
        while (not done):  # 최종 목적지 [9,4] 도착
            mov += 1
            # 상태 state1에서 action 결정
            #####################################################################################
            if test_env.curloc == [9,4]:  # 출발점에서는 무조건 위로 (action=0)
                action = 0
            elif test_env.curloc[0] == 0 and test_env.curloc[1] in [0,1,2,3,4,5,6,7,8]:  # (0,0)~(0,8)에서는 무조건 아래로
                action = 1
            elif test_env.curloc[0] in [2,3,4,5] and test_env.curloc[1] == 8:  # (2,8)~(5,8)에서는 무조건 왼쪽으로
                action = 2
            elif test_env.curloc[0] in [2,3,4,5] and test_env.curloc[1] == 0:  # (2,0)~(5,0)에서는 무조건 오른쪽으로
                action = 3
            else:  # 위 경우를 제외하곤 test 액션 선택
                action = test_q.test_action(history)
            #####################################################################################
            # action = test_q.test_action(history)  # 위 조건을 적용하지 않을 경우
            # make Move... action에 따른 이동
            gridmap, reward, cum_reward, done = test_env.step(action)
#             print(action, test_env.get_current_state(), reward, cum_reward)

            next_gridmap_ = np.reshape([gridmap],(1,1,10,9)) # 다음 그리드를 히스토리 사이즈에 맞게 reshape
            next_gridmap = torch.from_numpy(next_gridmap_).float() # 텐서로 변환
            new_history = torch.cat([next_gridmap, history[:,:3,:,:]],dim=1) # (1,1+3,10,9)
            # 지금 있는 히스토리중 마지막 채널을 지우고 new를 제일 앞에 붙임
            history = new_history

            if display:
                print('Move #%s; Taking action: %s' % (mov, action))
#                 print(gridmap.reshape(10,9))

            if test_env.goal_ob_reward:
                wins += 1
                if display:
                    print("Game won! Reward: %s" % (cum_reward))
                    
            if (mov > 250):  # 경로 길이가 너무 길어서 제외시키는 조건 추가 필요시 사용
                if display:
                    print("Game lost; too many moves.")
                break
                
        ##### gif 생성을 위한 경로(env.actions) 저장 ####################################    
        f.write(str(n_epi)+'/'+str(test_env.local_target_original)+'\n')
        f.write(str(test_env.actions))
        f.write('\n')
        ######################################### 
    f.close()
                
    win_perc = float(wins) / float(epochs)
    print("Test performed: {0}, # of wins: {1}".format(epochs,wins))
    print("Win percentage: {}%".format(100.0*win_perc))
    return win_perc

tensor in cuda


In [5]:
test(epochs=1226, train_mode=False, display=True, model_load=True, PATH = PATH)

data/factory_order_test.csv used
Move #1; Taking action: 0
Move #2; Taking action: 0
Move #3; Taking action: 2
Move #4; Taking action: 2
Move #5; Taking action: 2
Move #6; Taking action: 0
Move #7; Taking action: 0
Move #8; Taking action: 2
Move #9; Taking action: 3
Move #10; Taking action: 0
Move #11; Taking action: 2
Move #12; Taking action: 3
Move #13; Taking action: 0
Move #14; Taking action: 0
Move #15; Taking action: 2
Move #16; Taking action: 3
Move #17; Taking action: 3
Move #18; Taking action: 3
Move #19; Taking action: 0
Move #20; Taking action: 0
Move #21; Taking action: 1
Move #22; Taking action: 3
Move #23; Taking action: 3
Move #24; Taking action: 0
Move #25; Taking action: 1
Move #26; Taking action: 3
Move #27; Taking action: 3
Move #28; Taking action: 3
Move #29; Taking action: 0
Move #30; Taking action: 1
Move #31; Taking action: 2
Move #32; Taking action: 1
Move #33; Taking action: 1
Move #34; Taking action: 1
Move #35; Taking action: 3
Move #36; Taking action: 2
Move

Move #14; Taking action: 2
Move #15; Taking action: 3
Move #16; Taking action: 0
Move #17; Taking action: 3
Move #18; Taking action: 0
Move #19; Taking action: 0
Move #20; Taking action: 1
Move #21; Taking action: 3
Move #22; Taking action: 3
Move #23; Taking action: 3
Move #24; Taking action: 0
Move #25; Taking action: 1
Move #26; Taking action: 3
Move #27; Taking action: 3
Move #28; Taking action: 0
Move #29; Taking action: 1
Move #30; Taking action: 1
Move #31; Taking action: 3
Move #32; Taking action: 2
Move #33; Taking action: 2
Move #34; Taking action: 2
Move #35; Taking action: 1
Move #36; Taking action: 1
Move #37; Taking action: 1
Move #38; Taking action: 1
Move #39; Taking action: 1
Move #40; Taking action: 1
Move #41; Taking action: 2
Move #42; Taking action: 1
Game won! Reward: 82.81
Move #1; Taking action: 0
Move #2; Taking action: 0
Move #3; Taking action: 2
Move #4; Taking action: 2
Move #5; Taking action: 2
Move #6; Taking action: 0
Move #7; Taking action: 0
Move #8; Ta

Move #24; Taking action: 0
Move #25; Taking action: 1
Move #26; Taking action: 3
Move #27; Taking action: 0
Move #28; Taking action: 1
Move #29; Taking action: 1
Move #30; Taking action: 3
Move #31; Taking action: 3
Move #32; Taking action: 3
Move #33; Taking action: 3
Move #34; Taking action: 2
Move #35; Taking action: 2
Move #36; Taking action: 2
Move #37; Taking action: 1
Move #38; Taking action: 1
Move #39; Taking action: 1
Move #40; Taking action: 1
Move #41; Taking action: 1
Move #42; Taking action: 1
Move #43; Taking action: 2
Move #44; Taking action: 1
Game won! Reward: 81.945
Move #1; Taking action: 0
Move #2; Taking action: 0
Move #3; Taking action: 2
Move #4; Taking action: 2
Move #5; Taking action: 2
Move #6; Taking action: 0
Move #7; Taking action: 0
Move #8; Taking action: 0
Move #9; Taking action: 0
Move #10; Taking action: 2
Move #11; Taking action: 3
Move #12; Taking action: 0
Move #13; Taking action: 2
Move #14; Taking action: 3
Move #15; Taking action: 3
Move #16; Ta

Move #39; Taking action: 1
Move #40; Taking action: 1
Move #41; Taking action: 2
Move #42; Taking action: 2
Move #43; Taking action: 2
Move #44; Taking action: 1
Game won! Reward: 81.005
Move #1; Taking action: 0
Move #2; Taking action: 2
Move #3; Taking action: 0
Move #4; Taking action: 2
Move #5; Taking action: 2
Move #6; Taking action: 0
Move #7; Taking action: 0
Move #8; Taking action: 0
Move #9; Taking action: 2
Move #10; Taking action: 3
Move #11; Taking action: 0
Move #12; Taking action: 0
Move #13; Taking action: 0
Move #14; Taking action: 2
Move #15; Taking action: 0
Move #16; Taking action: 1
Move #17; Taking action: 3
Move #18; Taking action: 0
Move #19; Taking action: 1
Move #20; Taking action: 3
Move #21; Taking action: 3
Move #22; Taking action: 3
Move #23; Taking action: 3
Move #24; Taking action: 0
Move #25; Taking action: 1
Move #26; Taking action: 3
Move #27; Taking action: 3
Move #28; Taking action: 0
Move #29; Taking action: 1
Move #30; Taking action: 1
Move #31; Ta

Move #14; Taking action: 0
Move #15; Taking action: 1
Move #16; Taking action: 3
Move #17; Taking action: 0
Move #18; Taking action: 1
Move #19; Taking action: 3
Move #20; Taking action: 3
Move #21; Taking action: 3
Move #22; Taking action: 0
Move #23; Taking action: 1
Move #24; Taking action: 3
Move #25; Taking action: 0
Move #26; Taking action: 1
Move #27; Taking action: 1
Move #28; Taking action: 3
Move #29; Taking action: 1
Move #30; Taking action: 3
Move #31; Taking action: 2
Move #32; Taking action: 1
Move #33; Taking action: 3
Move #34; Taking action: 2
Move #35; Taking action: 1
Move #36; Taking action: 1
Move #37; Taking action: 1
Move #38; Taking action: 1
Move #39; Taking action: 2
Move #40; Taking action: 2
Move #41; Taking action: 2
Move #42; Taking action: 1
Game won! Reward: 82.82
Move #1; Taking action: 0
Move #2; Taking action: 0
Move #3; Taking action: 2
Move #4; Taking action: 2
Move #5; Taking action: 2
Move #6; Taking action: 0
Move #7; Taking action: 0
Move #8; Ta

...

Move #8; Taking action: 2
Move #9; Taking action: 3
Move #10; Taking action: 0
Move #11; Taking action: 0
Move #12; Taking action: 2
Move #13; Taking action: 3
Move #14; Taking action: 0
Move #15; Taking action: 3
Move #16; Taking action: 0
Move #17; Taking action: 0
Move #18; Taking action: 1
Move #19; Taking action: 3
Move #20; Taking action: 3
Move #21; Taking action: 3
Move #22; Taking action: 0
Move #23; Taking action: 1
Move #24; Taking action: 3
Move #25; Taking action: 3
Move #26; Taking action: 3
Move #27; Taking action: 0
Move #28; Taking action: 1
Move #29; Taking action: 2
Move #30; Taking action: 1
Move #31; Taking action: 1
Move #32; Taking action: 1
Move #33; Taking action: 3
Move #34; Taking action: 2
Move #35; Taking action: 1
Move #36; Taking action: 3
Move #37; Taking action: 2
Move #38; Taking action: 1
Move #39; Taking action: 1
Move #40; Taking action: 1
Move #41; Taking action: 2
Move #42; Taking action: 2
Move #43; Taking action: 2
Move #44; Taking action: 1
Gam

Move #29; Taking action: 1
Move #30; Taking action: 3
Move #31; Taking action: 1
Move #32; Taking action: 1
Move #33; Taking action: 1
Move #34; Taking action: 3
Move #35; Taking action: 2
Move #36; Taking action: 1
Move #37; Taking action: 1
Move #38; Taking action: 1
Move #39; Taking action: 2
Move #40; Taking action: 2
Move #41; Taking action: 2
Move #42; Taking action: 1
Game won! Reward: 82.83999999999999
Move #1; Taking action: 0
Move #2; Taking action: 2
Move #3; Taking action: 2
Move #4; Taking action: 2
Move #5; Taking action: 0
Move #6; Taking action: 0
Move #7; Taking action: 0
Move #8; Taking action: 0
Move #9; Taking action: 0
Move #10; Taking action: 0
Move #11; Taking action: 2
Move #12; Taking action: 3
Move #13; Taking action: 0
Move #14; Taking action: 0
Move #15; Taking action: 1
Move #16; Taking action: 3
Move #17; Taking action: 3
Move #18; Taking action: 3
Move #19; Taking action: 0
Move #20; Taking action: 1
Move #21; Taking action: 3
Move #22; Taking action: 3
M

Move #1; Taking action: 0
Move #2; Taking action: 2
Move #3; Taking action: 0
Move #4; Taking action: 2
Move #5; Taking action: 2
Move #6; Taking action: 0
Move #7; Taking action: 0
Move #8; Taking action: 0
Move #9; Taking action: 0
Move #10; Taking action: 2
Move #11; Taking action: 3
Move #12; Taking action: 0
Move #13; Taking action: 2
Move #14; Taking action: 3
Move #15; Taking action: 0
Move #16; Taking action: 0
Move #17; Taking action: 1
Move #18; Taking action: 3
Move #19; Taking action: 3
Move #20; Taking action: 0
Move #21; Taking action: 1
Move #22; Taking action: 3
Move #23; Taking action: 0
Move #24; Taking action: 1
Move #25; Taking action: 3
Move #26; Taking action: 0
Move #27; Taking action: 1
Move #28; Taking action: 1
Move #29; Taking action: 3
Move #30; Taking action: 3
Move #31; Taking action: 1
Move #32; Taking action: 1
Move #33; Taking action: 3
Move #34; Taking action: 2
Move #35; Taking action: 1
Move #36; Taking action: 1
Move #37; Taking action: 1
Move #38; 

Move #23; Taking action: 3
Move #24; Taking action: 0
Move #25; Taking action: 1
Move #26; Taking action: 3
Move #27; Taking action: 0
Move #28; Taking action: 1
Move #29; Taking action: 3
Move #30; Taking action: 0
Move #31; Taking action: 1
Move #32; Taking action: 1
Move #33; Taking action: 3
Move #34; Taking action: 2
Move #35; Taking action: 2
Move #36; Taking action: 2
Move #37; Taking action: 1
Move #38; Taking action: 1
Move #39; Taking action: 1
Move #40; Taking action: 1
Move #41; Taking action: 1
Move #42; Taking action: 1
Move #43; Taking action: 2
Move #44; Taking action: 1
Game won! Reward: 80.815
Move #1; Taking action: 0
Move #2; Taking action: 0
Move #3; Taking action: 2
Move #4; Taking action: 2
Move #5; Taking action: 2
Move #6; Taking action: 0
Move #7; Taking action: 0
Move #8; Taking action: 2
Move #9; Taking action: 3
Move #10; Taking action: 0
Move #11; Taking action: 2
Move #12; Taking action: 3
Move #13; Taking action: 0
Move #14; Taking action: 0
Move #15; Ta

1.0